In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
import xlwings as xw
import datetime as dt
import holidays
import calendar

In [2]:
# import 1000 most common last names
url = 'https://names.mongabay.com/data/1000.html'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
last_names = list(df[0]) + ["Jameson"]
del last_names[0]

In [3]:
# import ~4500 most common female first names
url_f = 'https://names.mongabay.com/female_names_alpha.htm'
html_f = requests.get(url_f).content
df_list_f = pd.read_html(html_f)
df_f = df_list_f[0]

In [4]:
# import ~1200 most common male first names
url_m = 'https://names.mongabay.com/male_names_alpha.htm'
html_m = requests.get(url_m).content
df_list_m = pd.read_html(html_m)
df_m = df_list_m[0]

In [5]:
# create long list of female names reflecting the frequency of each name mutiplied by 1000
names_f = list(df_f["Name"])
frequency_f = list(df_f["%Frequency"])
names_f = [[names_f[i]]*round(frequency_f[i]*1000) for i in range(len(frequency_f))]
large_names_f = [j for i in names_f for j in i]
# create a random length-500 list of female names 
names_f = [random.choice(large_names_f) for i in range(500)]

In [6]:
# create long list of male names reflecting the frequency of each name mutiplied by 1000
names_m = list(df_m["Name"])
frequency_m = list(df_m["%Frequency"])
names_m = [[names_m[i]]*round(frequency_m[i]*1000) for i in range(len(frequency_m))]
large_names_m = [j for i in names_m for j in i]
# create a random length-500 list of male names 
names_m = [random.choice(large_names_m) for i in range(500)]

In [7]:
# combine the the two first name lists, then combine first and last names and put into dataframe
first_names = names_f + names_m
names = [random.choice(first_names) + ' ' + random.choice(last_names) for i in range(1000)]

In [8]:
# create batch IDs in the format YYYYYMMDD-######-FLLL 

# start with year2018: a list of 2018 work dates (eliminate all weekends and holidays) 

# designate 2018 holidays (we wil have to convert the holiday list to datetime from date)
us_holidays = holidays.UnitedStates()
dt.date(2018,1,1) in us_holidays
us_holidays = list(us_holidays)
us_holidays = [dt.datetime.combine(i, dt.time.min) for i in us_holidays]    

# create list of all 2018 days
year2018 = [dt.datetime(2018,i,j) for i in [1,3,5,7,8,10,12] for j in range(1,32)]
year2018 += [dt.datetime(2018,i,j) for i in [4,6,9,11] for j in range(1,31)]
year2018 += [dt.datetime(2018,2,j) for j in range(1,29)]
year2018.sort()

# designate weekends
weekends = [i for i in year2018 if dt.datetime.isoweekday(i) in [6,7]]

# combine holidays and weekends to create offdays
offdays = us_holidays + weekends

# iterate through year2018 to disclude offdays, giving 251 eligibe work dates
year2018 = [i for i in year2018 if i not in offdays]

# create list of 1000 dates
scan_date = [random.choice(year2018) for i in range(1000)]

In [9]:
# fetch dates from scan_date
scan_date = [i.date() for i in scan_date]

In [41]:
# format ids as YYYYMMDD-######-FLLL
users = [names[i][0] + names[i].split()[-1][:3] for i in range(len(names))]

test = [i.strftime("%Y%m%d") for i in scan_date]
batch_id = [f"{test[i]}-{random.randrange(1000,9999)}-{users[i]}" for i in range(1000)]

In [42]:
# create mock list of errors
def rand_str():
    errors_local = ['Compliance warning','Data breach','No agent name','No department listed', 'No mailing address', 'Wrong workflow']
    result = [random.choice(errors_local) for i in range(1000)]
    return result

errors = list(rand_str())

In [43]:
# create master data frame
df_master = pd.DataFrame()
df_master["Name"] = names
df_master['Batch ID'] = batch_id
df_master["Date"] = scan_date
df_master['Error'] = errors

In [ ]:
# For loading into Excel

# Option 1: For interactive connection to an excel workbook, use xlwings
wb = xw.Book('tracking.xlsm')
sheet = wb.sheets['Sheet1']

# set the dataframe to column A of excel sheet
sheet.range("A1").value = df_master

#Option 2: To export the completed dataframe to an excel sheet
df_master.to_excel('/Users/chrishuhn/desktop/tracking.xlsx')

In [44]:
df_master

,Name,Batch ID,Date,Error
0,YOLANDA RAMOS,20180711-1152-YRAM,2018-07-11,No mailing address
1,FLORENCE WOODWARD,20180914-7677-FWOO,2018-09-14,No department listed
2,MIKE RIVERS,20180328-4829-MRIV,2018-03-28,Wrong workflow
3,BRENDA GONZALES,20180104-9933-BGON,2018-01-04,No mailing address
4,MICHAEL HUANG,20180516-5808-MHUA,2018-05-16,Wrong workflow
5,EMIL VALENTINE,20180124-1684-EVAL,2018-01-24,No department listed
6,MILDRED WILEY,20180731-3443-MWIL,2018-07-31,No department listed
7,WAYNE THOMAS,20180416-8719-WTHO,2018-04-16,No department listed
8,MERCEDES BARNES,20180410-8235-MBAR,2018-04-10,No department listed
9,CLIFFORD BATES,20180412-6698-CBAT,2018-04-12,Wrong workflow
